# Dataset en Verwerking

Voor deze opdracht maken wij gebruik van drie verschillende datasets van verschillende bronnen. Deze worden in dit onderdeel beschreven en verwerkt. Ten slotte worden de datasets samengevoegd tot één dataset.

Voor deze opdracht maken wij gebruik van de volgende datasets:
- Persvrijheid dataset van [Reporters Without Borders (RSF) (2024)](./referenties) waarvan alleen data wordt gebruikt uit 2019 en 2023.
- Democratie dataset van [Our World In Data (2024)](./referenties) waarvan alleen data wordt gebruikt uit 2019 en 2023.
- Conflict Index dataset van [ACLED (2024)](./referenties) hier is alleen data van 2019 en 2023 beschikbaar.

In [1]:
import pandas as pd

from plotly.offline import init_notebook_mode
from IPython.display import display

init_notebook_mode(connected=True)

## Persvrijheid dataset

Voor de persvrijheid data maken we gebruik van de datasets van Reporters Without Borders (RSF) [(2024)](./referenties). Deze datasets bevatten informatie over de persvrijheid in verschillende landen, waaronder de persvrijheidsscore en de ranglijst van elk land. De datasets die wij gebruiken zijn van 2019 en 2023.

### Persvrijheid Dataset Schoonmaken

De persvrijheid dataset [(2024)](./referenties) van 2019 verschilt met die van 2023, dus worden deze eerst apart ingeladen en schoongemaakt. Daarna worden deze samengevoegd.

De persvrijheid dataset bevat 16 variabelen voor 2019 en 25 variabelen voor 2023, waarvan een aantal niet relevant zijn voor deze opdracht, zoals de namen van landen in verschillende talen. De variabelen die niet relevant zijn worden verwijderd.

De variabelen uit de persvrijheid dataset die gebruikt worden bij deze opdracht zijn:
- **iso**: _(Discreet / Nominaal)_ Een drieletterige code voor een land, volgens de ISO 3166-1 standaard.
- **land**: _(Discreet / Nominaal)_ De Engelse naam van het land.
- **rang**: _(Continu / Ratio)_ De rang van het land op basis van de persvrijheidsscore.
- **score**: _(Continu / Interval)_ De persvrijheidsscore van het land.

De variabelen `rang` en `score` zijn opgedeeld in de jaren waarvoor de data is, en worden bijvoorbeeld `rang_2019` en `score_2023`. 

#### 2019 Dataset

In [2]:
persvrijheid_oud = pd.read_csv('./data/persvrijheid/2019.csv', sep=';')

# Hernoemen van kolommen om deze makkelijker te kunnen gebruiken.
persvrijheid_oud = persvrijheid_oud.rename(
    columns={
        "Year (N)": "jaar",
        "ISO": "iso",
        "EN_country": "land",
        "Rank N": "rang",
        "Score N": "score",
    })

# Verwijderen van onnodige kolommen.
persvrijheid_oud = persvrijheid_oud.drop(columns=[
    "Score N without the exactions",
    "Score N with the exactions",
    "Rank N-1",
    "Rank evolution",
    "Score exactions",
    "Score N-1",
    "Zone", # De data uit deze kolom is niet door alle jaren heen hetzelfde, dus deze wordt verwijderd.
    "jaar",
    "land", # We maken gebruik van de 2023 landnamen.
])

# Verwijder kolommen met buitenlandse namen van landen.
persvrijheid_oud = persvrijheid_oud.drop(columns=[
    col
    for col in persvrijheid_oud.columns
    if "country" in col.lower()
])

display(persvrijheid_oud.head())

,iso,rang,score
0,NOR,1,"92,18"
1,FIN,2,"92,1"
2,SWE,3,"91,69"
3,NLD,4,"91,37"
4,DNK,5,"90,13"


#### 2023 Dataset

In [3]:
persvrijheid_nieuw = pd.read_csv('./data/persvrijheid/2023.csv', sep=';')

# Hernoemen van kolommen om deze makkelijker te kunnen gebruiken.
persvrijheid_nieuw = persvrijheid_nieuw.rename(
    columns={
        "Year (N)": "jaar",
        "ISO": "iso",
        "Country_EN": "land",
        "Rank": "rang",
        "Score": "score",
    })

# Verwijderen van onnodige kolommen.
persvrijheid_nieuw = persvrijheid_nieuw.drop(columns=[
    "Political Context", "Rank_Pol",
    "Economic Context", "Rank_Eco",
    "Legal Context", "Rank_Leg",
    "Social Context", "Rank_Soc",
    "Safety", "Rank_Saf",
    "Rank N-1",
    "Rank evolution",
    "Score N-1",
    "Score evolution",
    "Zone", # De data uit deze kolom is niet door alle jaren heen hetzelfde, dus deze wordt verwijderd.
    "jaar",
])

# Verwijder kolommen met buitenlandse namen van landen.
persvrijheid_nieuw = persvrijheid_nieuw.drop(columns=[
    col
    for col in persvrijheid_nieuw.columns
    if "country" in col.lower()
])

display(persvrijheid_nieuw.head())

,iso,score,rang,land
0,NOR,"95,18",1,Norway
1,IRL,"89,91",2,Ireland
2,DNK,"89,48",3,Denmark
3,SWE,"88,15",4,Sweden
4,FIN,"87,94",5,Finland


### Persvrijheid Dataset Samenvoegen

Door het samenvoegen van de twee datasets, ontstaat een dataset met 6 kolommen en 178 rijen.

In [4]:
# Datasets samenvoegen op basis van de ISO code van het land.
persvrijheid_df = pd.merge(
    persvrijheid_oud,
    persvrijheid_nieuw,
    on="iso",
    how="inner",
    suffixes=('_2019', '_2023')
)

# Herordenen van de kolommen. (Geen functionele waarde)
persvrijheid_df = persvrijheid_df[[
    "iso",
    "land",
    "score_2019",
    "rang_2019",
    "score_2023",
    "rang_2023",
]]

display(persvrijheid_df.head())
print(f"De persvrijheid dataset bevat {len(persvrijheid_df.columns)} variabelen en {len(persvrijheid_df)} rijen.")
print(f"In totaal zijn dat {len(persvrijheid_df.columns) * len(persvrijheid_df)} datapunten.")

,iso,land,score_2019,rang_2019,score_2023,rang_2023
0,NOR,Norway,"92,18",1,"95,18",1
1,FIN,Finland,"92,1",2,"87,94",5
2,SWE,Sweden,"91,69",3,"88,15",4
3,NLD,Netherlands,"91,37",4,87,6
4,DNK,Denmark,"90,13",5,"89,48",3


De persvrijheid dataset bevat 6 variabelen en 178 rijen.
In totaal zijn dat 1068 datapunten.


## Democratie dataset

Voor de democratie data maken we gebruik van de datasets van [Our World in Data (2024)](./referenties). Deze datasets bevatten informatie over de democratie in verschillende landen, waaronder de democratie score en de ranglijst van elk land. De dataset bevat data van 2006 tot 2023, maar wij gebruiken alleen de data van 2019 en 2023.

De variabelen uit de democratie dataset die gebruikt worden bij deze opdracht zijn:
- **iso**: _(Discreet / Nominaal)_ Een drieletterige code voor een land, volgens de ISO 3166-1 standaard.
- **land**: _(Discreet / Nominaal)_ De Engelse naam van het land.
- **score**: _(Continu / Interval)_ De democratie score van het land.
- **rang**: _(Continu / Ratio)_ De rang van het land tegenover andere landen op basis van de democratie score.

De variabelen `score` zijn opgedeeld in de jaren waarvoor de data is, en worden bijvoorbeeld `score_2019` en `score_2023`.


### Democratie Dataset Schoonmaken

In [5]:
democratie_df = pd.read_csv('./data/democracy.csv')

# Hernoemen van de kolommen om deze makkelijker te kunnen gebruiken.
democratie_df = democratie_df.rename(
    columns={
        "Entity": "land",
        "Code": "iso",
        "Year": "jaar",
        "Democracy score": "score",
    })

# Verwijder landen waarvan geen ISO code bekend is.
# Dit zijn geen landen, maar regio's.
democratie_df = democratie_df.dropna(subset=["iso"])

# Verwijder landen waarvan de ISO code geen ISO3 code is.
# Dit zijn in deze dataset alleen de gemiddelde scores van de wereld.
democratie_df = democratie_df[democratie_df["iso"].apply(lambda x: len(x) == 3)]

# Dataset herinrichten in kolommen voor scores van 2019 en 2023.
democratie_df = pd.merge(
    democratie_df[democratie_df["jaar"] == 2019],
    democratie_df[democratie_df["jaar"] == 2023],
    on="iso",
    how="inner",
    suffixes=('_2019', '_2023')
)

# Verwijderen van onnodige kolommen.
democratie_df = democratie_df.drop(columns=[
    "jaar_2019",
    "jaar_2023",
    "land_2019",
])

# Hernoemen van de 'land_2023' kolom naar 'land'.
democratie_df = democratie_df.rename(columns={"land_2023": "land"})

# Herordenen van de kolommen. (Geen functionele waarde)
democratie_df = democratie_df[[
    "iso",
    "land",
    "score_2019",
    "score_2023",
]]

display(democratie_df.head())

,iso,land,score_2019,score_2023
0,ALB,Albania,5.89,6.28
1,DZA,Algeria,4.01,3.66
2,AGO,Angola,3.72,4.18
3,ARG,Argentina,7.02,6.62
4,ARM,Armenia,5.54,5.42


### Democratie Rangschikken

Ieder land krijgt een ranking gebaseerd op de democratie score. Landen met de hoogste score krijgen de rang `1`. Landen met dezelfde score krijgen dezelfde ranking. De rangschikking voor ieder land wordt per jaar berekend.

In [6]:
def rangschik_democratie(df):
    new_df = df.copy()

    # Maak een geordende lijst van unieke scores.
    # Op basis van de index van een score in deze lijst wordt de rang bepaald.
    unieke_dem_scores_2019 = sorted(list(df["score_2019"].unique()), reverse=True)
    unieke_dem_scores_2023 = sorted(list(df["score_2023"].unique()), reverse=True)
    
    # Bereken de rangschikking van de landen.
    new_df["rang_2019"] = new_df["score_2019"].apply(lambda x: unieke_dem_scores_2019.index(x) + 1)
    new_df["rang_2023"] = new_df["score_2023"].apply(lambda x: unieke_dem_scores_2023.index(x) + 1)
    
    return new_df

democratie_df = rangschik_democratie(democratie_df)

display(democratie_df.head())
print(f"De democratie dataset bevat {len(democratie_df.columns)} variabelen en {len(democratie_df)} rijen.")
print(f"In totaal zijn dat {len(democratie_df.columns) * len(democratie_df)} datapunten.")

,iso,land,score_2019,score_2023,rang_2019,rang_2023
0,ALB,Albania,5.89,6.28,71,58
1,DZA,Algeria,4.01,3.66,102,100
2,AGO,Angola,3.72,4.18,107,97
3,ARG,Argentina,7.02,6.62,44,46
4,ARM,Armenia,5.54,5.42,78,75


De democratie dataset bevat 6 variabelen en 148 rijen.
In totaal zijn dat 888 datapunten.


## Conflict Index dataset

Voor de conflict data maken we gebruik van de datasets van [The Armed Conflict Location & Event Data Project (ACLED) (2024)](./referenties). Deze datasets bevatten informatie over conflicten in verschillende landen, waaronder de conflict rangschikking van elk land. De dataset die wij gebruiken heeft data vanaf het jaar 2023, en vergelijkt de data van een land met de data van het land uit 2019. Uit de vergelijkingen met 2019 worden de conflict index scores van 2019.

In de dataset wordt de rangschikking van een land uit 2019 bepaald door de rangschikking van 2023 en de verandering van de rangschikking sinds 2019.

De variabelen uit de conflict index dataset die gebruikt worden bij deze opdracht zijn:
- **land**: _(Discreet / Nominaal)_ De Engelse naam van het land.
- **rang_verandering**: _(Continu / Interval)_ De verandering van de rang van het land ten opzichte van 2019.
- **categorie**: _(Discreet / Nominaal)_ De categorie van het land op basis van de conflict index score.
- **rang_2019**: _(Continu / Ratio)_ De rang van het land op basis van de conflict index scores uit 2019.
- **rang_2023**: _(Continu / Ratio)_ De rang van het land op basis van de conflict index scores uit 2023.

De variabelen `rang` zijn net zoals de andere datasets opgedeeld in de jaren waarvoor de data is, en worden bijvoorbeeld `rang_2019` en `rang_2023`.

### Conflict Index Dataset Schoonmaken

In [12]:
conflict_df = pd.read_csv('./data/conflict.csv')

# Hernoemen van kolommen om deze makkelijker te kunnen gebruiken.
conflict_df = conflict_df.rename(columns={
    "country": "land",
    "Index Ranking 2023": "rang_2023",
    "Change in ranking since 2019": "rang_verandering",
    "Index Category 2023": "categorie_2023",
})


# Verwijderen van onnodige kolommen.
conflict_df = conflict_df[[
    "land",
    "rang_2023",
    "rang_verandering",
    "categorie_2023",
]]

# Berekenen van de rangschikking van 2019.
conflict_df["rang_2019"] = conflict_df["rang_2023"] + conflict_df["rang_verandering"]

### Conflict Index Rangschikken

In de Conflict Index dataset is al een rangschikking aanwezig, maar deze rangschikking geeft een land met een hoge mate van conflicten en/of geweld een hoge rang. Het land met de hoogste maat van conflict en/of geweld krijgt namelijk de rang van `1`. Omdat in deze opdracht bij de andere datasets een hoge rang een positieve score betekent, wordt de rangschikking van de Conflict Index dataset omgedraaid. Een land met de laagste conflict index rang krijgen de rang `1`. De rangschikking voor ieder land wordt per jaar berekend.

De formule voor het omdraaien van de rangschikking is hieronder weergegeven, waarbij $R$ voor de rang van een land staat.

$$R_{nieuw} = R_{max} - R_{oud} + 1$$

In [13]:
def rangschik_conflict(df):
    new_df = df.copy()

    # Omdraaien van de rangschikking.
    new_df["rang_2019"] = new_df["rang_2019"].max() - new_df["rang_2019"] + 1
    new_df["rang_2023"] = new_df["rang_2023"].max() - new_df["rang_2023"] + 1

    return new_df

conflict_df = rangschik_conflict(conflict_df)

display(conflict_df.head())

,land,rang_2023,rang_verandering,categorie_2023,rang_2019
0,Myanmar,163,17,Extreme,116
1,Syria,162,-1,Extreme,133
2,Palestine,161,20,Extreme,111
3,Mexico,160,-1,Extreme,131
4,Nigeria,159,6,Extreme,123


## Samenvoegen van datasets

De datasets van persvrijheid, democratie en conflict index worden samengevoegd tot één dataset. De datasets worden samengevoegd op basis van de ISO code van het land.

De rangen en/of scores van de datasets krijgen hierbij een prefix om aan te geven bij welke dataset de rang en/of score hoort, bijvoorbeeld `persvrijheid_rang_2023` voor de rang van de persvrijheid dataset van een land in 2023.

In [17]:
# Toevoegen van prefixes aan de kolomnamen, als deze er nog niet zijn.
def add_prefix(df, prefix):
    new_df = df.copy()
    for col in new_df.columns:
        if not col.startswith(prefix):
            new_df = new_df.rename(columns={col: prefix+col})
    return new_df

persvrijheid_df = add_prefix(persvrijheid_df, "persvrijheid_")
democratie_df = add_prefix(democratie_df, "democratie_")
conflict_df = add_prefix(conflict_df, "conflict_")

# Samenvoegen van de datasets.
eind_df = pd.merge(
    persvrijheid_df,
    democratie_df,
    left_on="persvrijheid_iso",
    right_on="democratie_iso",
    how="inner"
)

# Verwijderen van de dubbele kolommen.
eind_df = eind_df.drop(columns=[
    "democratie_iso",
    "democratie_land",
])

# Hernoemen van kolommen.
eind_df = eind_df.rename(columns={
    "persvrijheid_iso": "iso",
    "persvrijheid_land": "land",
})

# Sommige landen namen komen niet in alle datasets voor, of zijn niet hetzelfde.
# De landen die niet een conflict index hebben worden hier weergegeven.
# display(set(eind_df["land"]) - set(conflict_df["conflict_land"]))

# Dataset samenvoegen met de conflict dataset op basis van land.
eind_df = pd.merge(
    eind_df,
    conflict_df,
    left_on="land",
    right_on="conflict_land",
    how="inner"
)

# Verwijderen van de dubbele kolommen.
eind_df = eind_df.drop(columns=[
    "conflict_land",
])

display(eind_df.head())

,iso,land,persvrijheid_score_2019,persvrijheid_rang_2019,persvrijheid_score_2023,persvrijheid_rang_2023,democratie_score_2019,democratie_score_2023,democratie_rang_2019,democratie_rang_2023,conflict_rang_2023,conflict_rang_verandering,conflict_categorie_2023,conflict_rang_2019
0,NOR,Norway,"92,18",1,"95,18",1,9.87,9.81,1,1,18,-13,Low/Inactive,1
1,FIN,Finland,"92,1",2,"87,94",5,9.25,9.30,5,5,18,-13,Low/Inactive,1
2,SWE,Sweden,"91,69",3,"88,15",4,9.39,9.39,3,4,24,-7,Low/Inactive,1
3,NLD,Netherlands,"91,37",4,87,6,9.01,9.00,10,9,39,8,Low/Inactive,1
4,DNK,Denmark,"90,13",5,"89,48",3,9.22,9.28,7,6,1,-30,Low/Inactive,1


### Veranderingen

Deze kolommen worden gebruikt bij visualisaties om de verandering van de rangen in de verschillende datasets te visualiseren. De veranderingen worden berekend door de rang van 2019 af te trekken van de rang van 2023. De formule is hieronder weergegeven, waarbij $R$ voor de rang van een land staat.

$$R_{verandering} = R_{2019} - R_{2023}$$

Deze berekening wordt uiteindelijk gedaan voor de persvrijheid, democratie en conflict index datasets.

In [18]:
# Voeg 2 extra kolommen toe voor de rang veranderingen voor persvrijheid en democratie. 
eind_df['persvrijheid_verandering'] = eind_df['persvrijheid_rang_2019'] - eind_df['persvrijheid_rang_2023']
eind_df['democratie_verandering'] = eind_df['democratie_rang_2019'] - eind_df['democratie_rang_2023']

## Resultaat

Uiteindelijk bestaat de samengevoegde dataset uit 16 variabelen, met 139 rijen.

Doordat de drie datasets niet data bevatten over dezelfde landen en de landen niet altijd dezelfde naam hebben, zijn er landen die niet in de samengevoegde dataset voorkomen. Daarnaast zijn in de persvrijheid dataset regio's aanwezig dat geen landen zijn, waardoor het aantal verloren landen erg hoog lijkt.

In [38]:
maximum_aantal_landen = max(len(persvrijheid_df), len(democratie_df), len(conflict_df))
print(f"In totaal zijn er minimaal {maximum_aantal_landen - len(eind_df)} landen/regio's die niet in de samengevoegde dataset voorkomen.")

# Dataset info weergeven.
display(eind_df.head())
print(f"De uiteindelijke dataset bevat {len(eind_df.columns)} variabelen en {len(eind_df)} rijen.")
print(f"In totaal zijn dat {len(eind_df.columns) * len(eind_df)} datapunten.")

In totaal zijn er minimaal 104 landen/regio's die niet in de samengevoegde dataset voorkomen.


,iso,land,persvrijheid_score_2019,persvrijheid_rang_2019,persvrijheid_score_2023,persvrijheid_rang_2023,democratie_score_2019,democratie_score_2023,democratie_rang_2019,democratie_rang_2023,conflict_rang_2023,conflict_rang_verandering,conflict_categorie_2023,conflict_rang_2019,persvrijheid_verandering,democratie_verandering
0,NOR,Norway,"92,18",1,"95,18",1,9.87,9.81,1,1,18,-13,Low/Inactive,1,0,0
1,FIN,Finland,"92,1",2,"87,94",5,9.25,9.30,5,5,18,-13,Low/Inactive,1,-3,0
2,SWE,Sweden,"91,69",3,"88,15",4,9.39,9.39,3,4,24,-7,Low/Inactive,1,-1,-1
3,NLD,Netherlands,"91,37",4,87,6,9.01,9.00,10,9,39,8,Low/Inactive,1,-2,1
4,DNK,Denmark,"90,13",5,"89,48",3,9.22,9.28,7,6,1,-30,Low/Inactive,1,2,1


De uiteindelijke dataset bevat 16 variabelen en 139 rijen.
In totaal zijn dat 2224 datapunten.


## Dataset opslaan

De samengevoegde dataset wordt opgeslagen in Pickle bestand, zodat deze in andere notebooks gebruikt kan worden.

In [39]:
eind_df.to_pickle('./data/dataset.pkl')

print("De dataset is opgeslagen in 'data/dataset.pkl'.")